# Multi-View-Majority-Vote-Learning-Algorithms-Direct-Minimization-of-PAC-Bayesian-Bounds

This Notebook contains everything necessary to reproduce the experiments in our paper:  

*Multi-View Majority Vote Learning Algorithms: Direct Minimization of PAC-Bayesian Bounds*

## Imports

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict

# Scikit-learn
from sklearn import preprocessing
from sklearn.utils import check_random_state
RAND = check_random_state(42)

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


from mvpb.dNDF import MultiViewBoundsDeepNeuralDecisionForests


# Import data
from mvlearn.datasets import load_UCImultifeature
from data.datasets import (SampleData,
                           MultipleFeatures,
                           Nutrimouse,
                           train_test_split,
                           train_test_merge,
                           s1_s2_split)
from mvpb.util import uniform_distribution

## Load and prepare the multiview datasets

In [2]:
dataset = MultipleFeatures()
X_train, y_train, X_test, y_test = dataset.get_data()
np.unique(y_train)
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
Xs_train = []
Xs_test = []
for xtr, xts in zip(X_train, X_test):
    scaler = preprocessing.MinMaxScaler().fit(xtr)
    Xs_train.append(scaler.transform(xtr))
    Xs_test.append(scaler.transform(xts))

X_train_concat = np.concatenate(Xs_train, axis=1)
X_test_concat = np.concatenate(Xs_test, axis=1)

In [4]:
mvb = MultiViewBoundsDeepNeuralDecisionForests(nb_estimators=3,
                                               nb_views=len(Xs_train),
                                               depth =3,
                                               used_feature_rate=0.8,
                                               epochs=5)

In [5]:
mvb.fit(Xs_train,y_train)

View 1/6 done!
View 2/6 done!
View 3/6 done!
View 4/6 done!
View 5/6 done!
View 6/6 done!


MultiViewBoundsDeepNeuralDecisionForests(depth=3, epochs=5, nb_estimators=3,
                                         nb_views=6,
                                         posterior_rho=tensor([0.3333, 0.3333, 0.3333]),
                                         used_feature_rate=0.8)

In [6]:
posterior_Qv , posterior_rho = mvb.optimize_rho('TND')

Iteration: 0,	 Loss: 7.179305398798768
Iteration: 1,	 Loss: 7.175806330616396
Iteration: 2,	 Loss: 7.1691346817083925
Iteration: 3,	 Loss: 7.159561080686371
Iteration: 4,	 Loss: 7.1473115672042375
Iteration: 5,	 Loss: 7.132577468783711
Iteration: 6,	 Loss: 7.1155201580165155
Iteration: 7,	 Loss: 7.096268109358897
Iteration: 8,	 Loss: 7.074938125656178
Iteration: 9,	 Loss: 7.051624144468582
Iteration: 10,	 Loss: 7.026401345738715
Iteration: 11,	 Loss: 6.99933622445835
Iteration: 12,	 Loss: 6.970486961712528
Iteration: 13,	 Loss: 6.9398945096320706
Iteration: 14,	 Loss: 6.907601770532688
Iteration: 15,	 Loss: 6.873641292004048
Iteration: 16,	 Loss: 6.838044428282892
Iteration: 17,	 Loss: 6.800837024124917
Iteration: 18,	 Loss: 6.762046493495845
Iteration: 19,	 Loss: 6.721703112804063
Iteration: 20,	 Loss: 6.679832523427158
Iteration: 21,	 Loss: 6.636470830695725
Iteration: 22,	 Loss: 6.591655031272936
Iteration: 23,	 Loss: 6.545430346890178
Iteration: 24,	 Loss: 6.497853791148488
Iterati

In [7]:
mvb.mv_tandem_risk()

(array([0.74064456, 0.49109924, 0.66099695, 0.28726407, 0.72633212,
        0.86187777]),
 0.0945646005844038,
 178.0)

In [8]:
_, mv_risk = mvb.predict_MV(Xs_test,y_test)
mv_risk

0.42000000000000004

In [9]:
lamb_bound, lamb_views = mvb.bound(bound="TND")

self.posterior_rho=tensor([0.0092, 0.0244, 0.0090, 0.9420, 0.0083, 0.0071]),  prior_pi=tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
KL_rhopi=tensor(2.0260),  KL_QP=tensor(0.0970)


In [10]:
lamb_bound, lamb_views

(1.0, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [9]:
mvb.bound(bound="PBkl")

self.posterior_rho=tensor([0.0073, 0.0218, 0.0068, 0.9439, 0.0102, 0.0100]),  prior_pi=tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
KL_rhopi=tensor(2.0397),  KL_QP=tensor(0.0014)
len(emp_rv)=6


(1.0854702312784439,
 [1.7977170927578487,
  1.4219816433125103,
  1.8253453317948358,
  1.0262293666270264,
  1.7028249649815994,
  1.770332677328597])

In [11]:
print(posterior_Qv[0])
print(posterior_Qv[1])
print(posterior_Qv[2])
print(posterior_Qv[3])
print(posterior_Qv[4])
print(posterior_Qv[5])

tensor([0.3539, 0.3318, 0.3143])
tensor([0.3519, 0.2919, 0.3562])
tensor([0.3207, 0.3116, 0.3678])
tensor([0.3326, 0.3327, 0.3347])
tensor([0.3377, 0.3328, 0.3295])
tensor([0.3230, 0.3430, 0.3340])


In [12]:
posterior_rho

tensor([0.1537, 0.1496, 0.1604, 0.2218, 0.1695, 0.1449])